In [1]:
import Bio

In [49]:
filename = 'dna2.fasta'

In [55]:
from Bio import SeqIO

fasta_file = filename  # Replace with your file name

num_sequences = sum(1 for _ in SeqIO.parse(fasta_file, "fasta"))

print(f"Number of sequences: {num_sequences}")

Number of sequences: 18


In [56]:
from Bio import SeqIO

def get_fasta_lengths(fasta_file):
    """
    Reads a multi-FASTA file and returns a dictionary with sequence IDs as keys and sequence lengths as values.
    
    :param fasta_file: Path to the multi-FASTA file
    :return: Dictionary {sequence_id: sequence_length}
    """
    seq_lengths = {record.id: len(record.seq) for record in SeqIO.parse(fasta_file, "fasta")}
    return seq_lengths

# Example usage
fasta_file = filename  # Replace with your actual file
lengths = get_fasta_lengths(fasta_file)

# Print results
for seq_id, length in lengths.items():
    print(f"{seq_id}: {length} bp")


gi|142022655|gb|EQ086233.1|91: 4635 bp
gi|142022655|gb|EQ086233.1|304: 1151 bp
gi|142022655|gb|EQ086233.1|255: 4894 bp
gi|142022655|gb|EQ086233.1|45: 3511 bp
gi|142022655|gb|EQ086233.1|396: 4076 bp
gi|142022655|gb|EQ086233.1|250: 2867 bp
gi|142022655|gb|EQ086233.1|322: 442 bp
gi|142022655|gb|EQ086233.1|88: 890 bp
gi|142022655|gb|EQ086233.1|594: 967 bp
gi|142022655|gb|EQ086233.1|293: 4338 bp
gi|142022655|gb|EQ086233.1|75: 1352 bp
gi|142022655|gb|EQ086233.1|454: 4564 bp
gi|142022655|gb|EQ086233.1|16: 4804 bp
gi|142022655|gb|EQ086233.1|584: 964 bp
gi|142022655|gb|EQ086233.1|4: 2095 bp
gi|142022655|gb|EQ086233.1|277: 1432 bp
gi|142022655|gb|EQ086233.1|346: 115 bp
gi|142022655|gb|EQ086233.1|527: 2646 bp


In [57]:
from Bio import SeqIO

def get_fasta_lengths(fasta_file):
    """
    Reads a multi-FASTA file and returns a dictionary with sequence IDs as keys and sequence lengths as values.
    
    :param fasta_file: Path to the multi-FASTA file
    :return: Dictionary {sequence_id: sequence_length}
    """
    return {record.id: len(record.seq) for record in SeqIO.parse(fasta_file, "fasta")}

def find_longest_shortest_sequences(seq_lengths):
    """
    Finds the longest and shortest sequences from a dictionary of sequence lengths.
    Returns all sequences if there are ties.

    :param seq_lengths: Dictionary {sequence_id: sequence_length}
    :return: Tuple of two lists - (longest_sequences, shortest_sequences),
             each list contains (sequence_id, sequence_length) tuples.
    """
    if not seq_lengths:
        return [], []  # Return empty lists if input is empty

    max_length = max(seq_lengths.values())
    min_length = min(seq_lengths.values())

    longest_seqs = [(seq_id, length) for seq_id, length in seq_lengths.items() if length == max_length]
    shortest_seqs = [(seq_id, length) for seq_id, length in seq_lengths.items() if length == min_length]

    return longest_seqs, shortest_seqs

# Example usage
fasta_file = filename  # Replace with your actual file
seq_lengths = get_fasta_lengths(fasta_file)  # Get sequence lengths
longest, shortest = find_longest_shortest_sequences(seq_lengths)  # Find longest & shortest sequences

# Print results
print("Longest Sequences:")
for seq_id, length in longest:
    print(f"{seq_id}: {length} bp")

print("\nShortest Sequences:")
for seq_id, length in shortest:
    print(f"{seq_id}: {length} bp")


Longest Sequences:
gi|142022655|gb|EQ086233.1|255: 4894 bp

Shortest Sequences:
gi|142022655|gb|EQ086233.1|346: 115 bp


In [77]:
from Bio import SeqIO
import re

# def find_orfs(fasta_file, frame=1):
#     """
#     Identifies all open reading frames (ORFs) in each DNA sequence for a given reading frame.
    
#     :param fasta_file: Path to the multi-FASTA file.
#     :param frame: Reading frame (1, 2, or 3).
#     :return: Dictionary {sequence_id: list of (start_position, orf_length, orf_sequence)}.
#     """
#     orfs = {}

#     for record in SeqIO.parse(fasta_file, "fasta"):
#         seq_id = record.id
#         sequence = str(record.seq).upper()  # Convert to uppercase for consistency

#         # Adjust sequence for reading frame (1-based)
#         adjusted_seq = sequence[frame - 1:]
        
#         # Find all ORFs (sequences starting with ATG and ending with a stop codon)
#         matches = re.finditer(r'ATG(?:...)*?(?:TAA|TAG|TGA)', adjusted_seq)
        
#         orfs[seq_id] = [
#             (match.start() + frame, len(match.group()), match.group()) for match in matches
#         ]

#     return orfs

def find_orfs(fasta_file, frame):
    """
    Identifies all open reading frames (ORFs) in each DNA sequence for a given reading frame.
    
    :param fasta_file: Path to the multi-FASTA file.
    :param frame: Reading frame (1, 2, or 3).
    :return: Dictionary {sequence_id: list of (start_position, orf_length, orf_sequence)}.
    """
    orfs = {}

    for record in SeqIO.parse(fasta_file, "fasta"):
        seq_id = record.id
        sequence = str(record.seq).upper()  # Convert to uppercase for consistency

        # Adjust sequence for reading frame (1-based)
        adjusted_seq = sequence[frame - 1:]

        # Find all ATG positions that align with the correct frame
        start_positions = [m.start() for m in re.finditer(r'ATG', adjusted_seq) if m.start() % 3 == 0]

        found_orfs = []

        # Search for valid ORFs starting from each ATG
        for start in start_positions:
            # Find the closest in-frame stop codon of any type
            stop_positions = sorted(
                m.start() for m in re.finditer(r'TAA|TAG|TGA', adjusted_seq) 
                if m.start() > start and (m.start() - start) % 3 == 0
            )

            if stop_positions:
                stop = stop_positions[0]  # Take the closest stop codon
                orf_seq = adjusted_seq[start:stop + 3]  # Include stop codon
                found_orfs.append((start + frame, len(orf_seq), orf_seq))

        # Only store sequences with at least one valid ORF
        if found_orfs:
            orfs[seq_id] = found_orfs

    return orfs



def find_longest_orf(orfs_dict):
    """
    Identifies the longest ORF across all sequences based on ORFs detected.

    :param orfs_dict: Dictionary from `find_orfs()` with {sequence_id: list of ORFs}.
    :return: Tuple (sequence_id, max_orf_length).
    """
    longest_orf = ("", 0)  # (Sequence ID, Length)

    for seq_id, orf_list in orfs_dict.items():
        for _, orf_length, _ in orf_list:
            if orf_length > longest_orf[1]:
                longest_orf = (seq_id, orf_length)

    return longest_orf

def get_longest_orf_info(orfs_dict, sequence_id):
    """
    Finds the location, length, and sequence of the longest ORF for a given sequence ID.

    :param orfs_dict: Dictionary from `find_orfs()` with {sequence_id: list of ORFs}.
    :param sequence_id: The sequence ID to look up.
    :return: Tuple (start_position, orf_length, orf_sequence), or None if no ORFs found.
    """
    if sequence_id not in orfs_dict or not orfs_dict[sequence_id]:
        return None

    # Find the ORF with the maximum length
    longest_orf = max(orfs_dict[sequence_id], key=lambda x: x[1])

    return longest_orf  # (start_position, length, sequence)


# Example Usage:
fasta_file = filename  # Replace with your file
reading_frame = 3  # Set the reading frame (1, 2, or 3)

# Step 1: Identify all ORFs
orfs_dict = find_orfs(fasta_file, frame = reading_frame)

# Step 2: Find the longest ORF in the file
longest_seq_id, longest_orf_length = find_longest_orf(orfs_dict)
print(f"Longest ORF found in {longest_seq_id} with length {longest_orf_length} bp")

# Step 3: Get details of the longest ORF for a given sequence
if longest_seq_id:
    longest_orf_info = get_longest_orf_info(orfs_dict, longest_seq_id)
    if longest_orf_info:
        start_pos, length, orf_seq = longest_orf_info
        print(f"Longest ORF in {longest_seq_id}:")
        print(f"Start position: {start_pos}")
        print(f"Length: {length}")
        print(f"Sequence: {orf_seq}")


Longest ORF found in gi|142022655|gb|EQ086233.1|527 with length 1821 bp
Longest ORF in gi|142022655|gb|EQ086233.1|527:
Start position: 636
Length: 1821
Sequence: ATGAACAGCGGGGCGAGCAAGCCGCCGGCCGTCACGGGGTCCATCACGAGGGACAGCAGCGGAATGCCGATGATCGCGAATCCACCACCGAACGCGCCGCGCATGAACGCGATCACGAACACGCCGGCAAACGCGATCAGGATCGTGGCCAGCGTCAATTGCAGGCCCATCGCAGCAGGGGTCGCCATCACGACCTCCATGCCGGTTCGAATCGCGGCGTGGCGGACAGCCACGGAGCGGGTCGCACGCGCGGCATCGCCGCACGATGGATCCGGGTTGAACGCGTTGCACCCATGCTGCTTCTCCAATGAGGTACCGGGGCGATGCGGTACACCAACGCACCGCAGGCCGCATGGGCCGCACAAGCATTTCAGCCCCGGTACAATCGACTTGACGAAAGCAGAATGCACCGCCGTCTATCTCAGTGCAATTAAAACATTGACCTCGGTGCAATATTCATTGTTATCGGTGCAATCCATGTCGAATTCCGAATACCTGCAGTTGGCCGACGCGATCGCCGCCCAAATTGCCGACGGCACGCTCAGGCCGGGCGACCGCCTGCCTCCGCAGCGTCATTTCGCCGACCAGCATGCGATCGCCGCATCGACGGCGGGACGGGTTTACGCGGAACTGTTACGGCGCGGCCTTGTGGTCGGCGAAGTCGGCCGAGGCACTTTCGTGTCGGGTGAGACGCGACGCGGGGCCGCTGCGCCGGGCGAGCCGCGCGGCGTTCGGATCGATTTCGAGTTCAACTACCCGACCGTCCCGGCCCAGACCGCGTTGATCACCAGAAGCCTGCGCGGATTGCACCGACCTGCGGAGCTCGACGCCGCGTTAC

In [78]:
get_longest_orf_info(orfs_dict, 'gi|142022655|gb|EQ086233.1|16')

(1440,
 1644,
 'ATGCGGGCCATCCTGCATCGCCGCCTTTCGTTCCACCCGGGCCGGCATCGAGTGATGCCGGCGTTGACGTTTTCGTGGAGTGAGTCAGATGAATCACGCAGCGAATCCCGCCGATCCCGATCGCGCCGCGGCGCAGGGCGGCAGCCTGTACAACGACGATCTCGCGCCGACGACGCCGGCGCAGCGCACGTGGAAGTGGTATCACTTCGCGGCGCTGTGGGTCGGGATGGTGATGAACATCGCGTCGTACATGCTCGCGGCCGGGCTGATCCAGGAAGGCATGTCGCCGTGGCAGGCGGTGACGACGGTGCTGCTCGGCAACCTGATCGTGCTCGTGCCGATGCTGCTGATCGGCCATGCGGGCGCGAAGCACGGGATTCCGTACGCGGTGCTCGTGCGCGCGTCGTTCGGCACGCAGGGGGCGAAGCTGCCGGCGCTGCTGCGCGCGATCGTCGCGTGCGGCTGGTACGGGATCCAGACCTGGCTCGGCGGCAGCGCGATCTATACGCTGCTGAACATCCTGACCGGCAACGCGCTGCATGGCGCCGCGCTGCCGGTCATCGGCATCGGGTTCGGGCAGCTCGCATGCTTCCTCGTGTTCTGGGCGCTGCAGCTCTACTTCATCTGGCATGGCACCGATTCGATCCGCTGGCTCGAAAGCTGGTCGGCGCCGATCAAGGTCGTGATGTGCGTGGCGCTGGTGTGGTGGGCAACGTCGAAGGCGGGCGGCTTCGGCACGATGCTGTCGGCGCCGTCGCAGTTTGCCGCAGGCGGCAAGAAAGCCGGGCTGTTCTGGGCGACCTTCTGGCCGGGGCTGACCGCGATGGTCGGCTTCTGGGCGACGCTCGCGCTGAACATCCCCGACTTCACGCGCTTCGCGCATTCGCAGCGCGACCAGGTGATCGGCCAGTCGATCGGGCTGCCGTTGCCGATGGCGCTGCTGTCGGTGGTGTCGGTCGTCGTGACGTCGGCGACCGTCGTGAT

In [24]:
sequences = {record.id: str(record.seq).upper() for record in SeqIO.parse(fasta_file, "fasta")}
sequences

{'gi|142022655|gb|EQ086233.1|43': 'TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGGCACCGCGGGCCTCTGCCGTGCGCTGCTTGGCCATGGCCTCCAGCGCACCGATCGGATCAAAGCCGCTGAAGCCTTCGCGCATCAGGCGGCCATAGTTGGCGCCAGTGACCGTACCAACCGCCTTGATGCGGCGCTCGGTCATCGCTGCATTGATCGAGTAGCCACCGCCGCCGCAAATGCCCAGCACGCCAATGCGTTCTTCATCCACATAGGGGAGCGTTACGAGGTAGTCGCAGACCACGCGGAAATCCTCGACGCGCAGTGTCGGGTCTTCGGTAAAACGTGGTTCGCCGCCGCTGGCACCCTGGAAGCTGGCGTCGAAGGCGATGACGACGAAACCTTCCTTGGCCAGCGCCTCGCCATACACGTTCCCCGATGTTTGCTCCTTGCAGCTGCCGATCGGATGCGCGCTGATGATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCGGCGGGAAGTGGATGTCGGCTGCGATATCCCAATACACATTCTTGATCTTGACGCTTTTCATGACAGCTCCGTTCAGGGGGAGGGGGTAAGTTCGCCAGGCCGAATCGTTGGTAGCCAAGCGGCAACGACTCGAATATAGAGAGCCGATTGGAATTCCGTAAGATCGCAATCTGGACTACAGTGGTATCTTCAAATTGACAATGGCACCTACATGGATCCCTCACTGCTTCCGTCTCTCGCGTGGTTCGCCCACGTCGCACATCATCGTAGCTTCACGAAAGCGGCTGCGGAAATGGGCGTTTCTCGAGCAAACCTGTCGCAGAACGTGAAGGCGCTCGAACGCCGGTTGAACGTCAAGCTGCTGTATCGAACGACTCGCGACATGTCGCTGACCGAGGAGGGGCAGCGGCTCTACGAGGTGTGGTATCCCGCGCTGGTCGCGGTCGAGCGGACGGTCGACGCGCTGCACGAGGAG

In [25]:
get_longest_orf_info(orfs_dict, 'gi|142022655|gb|EQ086233.1|160')

(75,
 165,
 'ATGGAAATCCCCGCTCAATCTTTGGAGCAGGGATGCGGGGCGATCAAGATGGGGATGCGGGATGGGGGCGACGGTGTATTTCCGCCAGAAGATTTCGCCGCGGGAGCTCGCGGTGCGTACGTGCATGTTCAAACGCACGGTGCGCGCATGGCAGTGGCAGACTGA')

In [27]:
sequences['gi|142022655|gb|EQ086233.1|160'][74:84]

'ATGGAAATCC'

In [79]:
from Bio import SeqIO
from collections import defaultdict, Counter

def find_repeats(fasta_file, n=7):
    """
    Identifies repeated substrings of length n in each sequence of a multi-FASTA file.

    :param fasta_file: Path to the multi-FASTA file.
    :param n: Length of the repeat.
    :return: Dictionary {sequence_id: Counter({repeat: count, ...})} containing repeat counts.
    """
    repeat_dict = {}

    for record in SeqIO.parse(fasta_file, "fasta"):
        seq_id = record.id
        sequence = str(record.seq).upper()  # Convert to uppercase for consistency
        repeat_counts = Counter()

        # Scan the sequence for repeats of length n
        for i in range(len(sequence) - n + 1):
            repeat = sequence[i:i+n]
            repeat_counts[repeat] += 1

        # Keep only repeated sequences (count > 1)
        repeat_dict[seq_id] = {k: v for k, v in repeat_counts.items() if v > 1}

    return repeat_dict

def find_most_frequent_repeat(repeat_dict):
    """
    Identifies the most frequent repeat for each sequence.

    :param repeat_dict: Dictionary from `find_repeats()` {sequence_id: {repeat: count, ...}}.
    :return: Dictionary {sequence_id: (repeat, count)} with the most frequent repeat.
    """
    most_frequent = {}

    for seq_id, repeats in repeat_dict.items():
        if repeats:  # Ensure there are repeats
            most_common = max(repeats.items(), key=lambda x: x[1])  # Find the most repeated pattern
            most_frequent[seq_id] = most_common

    return most_frequent

# Example Usage
fasta_file = filename  # Replace with your actual file
repeat_length = 12  # Define the length of the repeat

# Step 1: Find all repeated sequences
repeat_counts = find_repeats(fasta_file, repeat_length)

# Step 2: Identify the most frequent repeat per sequence
most_frequent_repeats = find_most_frequent_repeat(repeat_counts)

# Print Results
print("Repeated Sequences:")
for seq_id, repeats in repeat_counts.items():
    print(f"{seq_id}: {repeats}")

print("\nMost Frequent Repeats:")
for seq_id, (repeat, count) in most_frequent_repeats.items():
    print(f"{seq_id}: {repeat} ({count} times)")


Repeated Sequences:
gi|142022655|gb|EQ086233.1|91: {'TCGTCGGCGATC': 2, 'CGCGGTTCCGCG': 2, 'CTGCCGAACATC': 2, 'TCGATCGCGTCG': 2, 'CGATCGCGTCGG': 2, 'CGACGCCGTCGA': 2}
gi|142022655|gb|EQ086233.1|304: {}
gi|142022655|gb|EQ086233.1|255: {'GCGTGGTCGGCG': 2, 'TGCTGATCCTGT': 2, 'GCTGATCCTGTT': 2, 'CTGATCCTGTTC': 2, 'TCGCGCTTCGCG': 2}
gi|142022655|gb|EQ086233.1|45: {'CATTCGCCATTC': 10, 'ATTCGCCATTCG': 10, 'TTCGCCATTCGC': 10, 'TCGCCATTCGCC': 10, 'CGCCATTCGCCA': 9, 'GCCATTCGCCAT': 9, 'CCATTCGCCATT': 9, 'CAGGATCAGCGT': 2, 'GCAGCGCGGCGA': 2, 'AGGCCGTCGGCG': 2, 'GGCCGTCGGCGA': 2, 'GCCGTCGGCGAT': 2, 'CCGTCGGCGATG': 2}
gi|142022655|gb|EQ086233.1|396: {'GATGCGCTGGCG': 2, 'ATGCGCTGGCGC': 2, 'TGCGCTGGCGCG': 2, 'GCGCTGGCGCGC': 2, 'GATCGTCGCGCC': 2, 'ATCGTCGCGCCG': 2, 'CACCTTCAACGG': 2, 'TGCTGCGCGACC': 2}
gi|142022655|gb|EQ086233.1|250: {'CGCCGCGTCGCG': 2, 'ACAGCGTCGCGA': 2, 'CAGCGTCGCGAG': 2, 'AGCGTCGCGAGC': 2, 'CCGCGTCGTCGC': 2}
gi|142022655|gb|EQ086233.1|322: {}
gi|142022655|gb|EQ086233.1|88: {'CGGCCGA

In [80]:
from collections import Counter
from Bio import SeqIO

def find_most_frequent_repeat(fasta_file, n):
    """
    Finds the most abundant repeat of length n across all sequences in the FASTA file.
    
    :param fasta_file: Path to the multi-FASTA file.
    :param n: Length of the repeat to search for.
    :return: The most frequent repeat and its count.
    """
    repeat_counts = Counter()

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq).upper()  # Convert to uppercase for consistency
        
        # Count all repeats of length n within this sequence
        for i in range(len(sequence) - n + 1):
            repeat = sequence[i:i + n]
            repeat_counts[repeat] += 1

    # Identify the most frequent repeat
    if repeat_counts:
        most_common_repeat, max_count = repeat_counts.most_common(1)[0]
        return most_common_repeat, max_count
    else:
        return None, 0  # No repeats found

# Example Usage
fasta_file = filename  # Replace with actual FASTA file
repeat_length = 6  # Change n to desired length

most_frequent_repeat, count = find_most_frequent_repeat(fasta_file, repeat_length)

if most_frequent_repeat:
    print(f"Most frequent repeat of length {repeat_length}: {most_frequent_repeat} (Count: {count})")
else:
    print(f"No repeats of length {repeat_length} found.")


Most frequent repeat of length 6: GCGCGC (Count: 153)
